In [ ]:
#importing the necessary libraries
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime
from datetime import timedelta
from scipy.optimize import fsolve # Solving non-linear system of equation
import math
sns.set()

In [ ]:
def sir_plot(ret,t1=np.arange(0,200)):
  S,I,R = ret.T
  fig = plt.figure(facecolor='w')
  ax = fig.add_subplot(111, axisbelow=True)
  ax.plot(t1, S, 'b', alpha=0.5, lw=2, label='Susceptible')
  ax.plot(t1, I, 'r', alpha=0.5, lw=2, label='Infected')
  ax.plot(t1, R, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
  ax.set_xlabel('days')
  ax.set_ylabel('Number')
  ax.yaxis.set_tick_params(length=0)
  ax.xaxis.set_tick_params(length=0)
  ax.grid(b=True, which='major', c='w', lw=2, ls='-')
  legend = ax.legend()
  legend.get_frame().set_alpha(0.5)
  for spine in ('top', 'right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)
  #plt.show()
  return(plt)
  

In [ ]:
def actual_pred_plot(ret,actual):

  S,I,R =ret.T

  t = np.arange(len(actual))
  fig = plt.figure(facecolor='w')
  ax = fig.add_subplot(111, axisbelow=True)
  ax.plot(t, I[0:len(t)], 'r', alpha=0.5, lw=2, label='Infected(predicted)')

  ax.plot(t, actual, 'y', alpha=0.5, lw=2, label='Infected(actual)')

  ax.set_xlabel('days')
  ax.set_ylabel('Number of Infected')
  ax.yaxis.set_tick_params(length=0)
  ax.xaxis.set_tick_params(length=0)
  ax.grid(b=True, which='major', c='w', lw=2, ls='-')
  legend = ax.legend()
  legend.get_frame().set_alpha(0.5)
  for spine in ('top', 'right', 'bottom', 'left'):
    ax.spines[spine].set_visible(False)
  return(plt)


In [ ]:
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt =((-beta/N * S * I))
    dIdt = (beta/N * S * I)  - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

In [ ]:
class model:
    def __init__(self,df,p,R0,t=np.arange(0,200),f=1.00,gamma=1/14):

      self.df = df
      self.p = p
      self.R0 = R0
      self.t = t
      self.f = f
      self.gamma = gamma

    def SIR(self):

        self.df.columns = ['date','cured','death','infected']
        i0 = self.df['infected'].iloc[0]
        r0 = (self.df['cured'].iloc[0] + self.df['death'].iloc[0])
        s0 = self.p * self.f - i0 - r0
        beta = self.R0 * self.gamma
        y0 = s0, i0, r0

        ret = odeint(deriv, y0, self.t, args=(self.p*self.f, beta,self.gamma))
        #output_plot = sir_plot(ret,self.t)
        #actual_pred_plot(ret,self.df.infected.values)
        return(ret)


In [ ]:
def state_model(path='/content/'):

  df = pd.read_csv(path+'covid_19_india.csv')
  df = df[df['State/UnionTerritory']!='Unassigned']
  df = df.dropna(subset=['State/UnionTerritory'])
  df_population = pd.read_csv(path+'population_india_census2011.csv')
  df = df[['Date','State/UnionTerritory','Cured','Deaths','Confirmed']]
  df['Confirmed'] = df['Confirmed']-df['Cured']-df['Deaths']
  df['Date'] = pd.to_datetime(df['Date'],format='%d/%m/%y')
  df = df[df['Date'] >=pd.datetime(2020,3,1)]

  df_model = pd.DataFrame()

  df_final = pd.DataFrame()

  for state in df['State/UnionTerritory'].unique():
  
    df_state = df[df['State/UnionTerritory']==state]
    if df_state['Confirmed'].max() <=30:
      continue
    population = df_population[df_population['State / Union Territory']==state]['Population'].iloc[0]
    df_state_R0 = pd.DataFrame()
    R0_array = np.linspace(0,20,num=20000)
    for R0 in R0_array.flatten():


      STATE = model(df_state.drop(['State/UnionTerritory'],axis=1),population,R0)
      ret = STATE.SIR()
      S,I,R = ret.T
      actual =  df_state['Confirmed'].values
      train_pct_index = int(0.8 * len(actual)) #fixing trainin
      #train_pct_index = df_state[df_state['Date']<= pd.datetime(2020,3,23)]
      mse = (np.square(I[:train_pct_index] - actual[:train_pct_index])).mean(axis=None)**(0.5)
      df_temp = pd.DataFrame([[state,R0,mse]],columns=['State','R0','mse'])
      df_state_R0 = df_state_R0.append(df_temp)
    
    rmse_min = df_state_R0.mse.min()

    R0_min = df_state_R0[df_state_R0['mse']==df_state_R0.mse.min()]['R0'].iloc[0]
    STATE = model(df_state.drop(['State/UnionTerritory'],axis=1),population,R0_min)
    ret = STATE.SIR()
    S,I,R = ret.T
    df_sir = pd.DataFrame({'S':S,'I':I,'R':R})
    df_sir.to_excel(path+state+'(SIR_values).xlsx',index=False)
    plt = actual_pred_plot(ret,df_state['Confirmed'].values)
    plt.savefig(path+state+'(Actual vs Prediction).png',dpi=100)
    plt1 = sir_plot(ret)
    plt1.savefig(path+state+'(SIR).png',dpi=100)
    rmse_test = (np.square(I[train_pct_index:len(actual)] - actual[train_pct_index:])).mean(axis=None)**(0.5)
    df_result = pd.DataFrame([[state,R0_min,rmse_min,rmse_test]],columns=['State','R0','RMSE(train)','RMSE(test)'])
    df_final = df_final.append(df_result)

  df_final.to_excel(path+"statewise_R0.xlsx",index=False)


In [ ]:

def main():
  path = '/content/' #Enter the project path here
  state_model(path)



In [ ]:
if __name__ == "__main__": 
    main()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  # Remove the CWD from sys.path while we load stuff.
